In [62]:
import pandas as pd
from datetime import datetime, timedelta
import calendar
from dateutil import parser

In [38]:
df = pd.read_csv("/Users/demafalih/python-bank-project-start/data/transactions.csv")

/var/folders/bj/1mckt7zn22xfvns4k5jdg5kr0000gn/T/ipykernel_78425/4199926100.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/demafalih/python-bank-project-start/data/transactions.csv")


---
### Tar bort mellanslag och dolda tecken:

In [39]:
for col in df.columns:
    if df[col].dtype == 'object':
       df[col] = df[col].astype(str)
       df[col] = df[col].str.replace(r"[\t\xa0]", "", regex=True)
       df[col] = df[col].str.strip() 

In [40]:
df['amount'] = df['amount'].str.replace(" ", "")
df['amount'] = df['amount'].astype(float)

In [41]:
print(df.head())
print(df.dtypes)

                         transaction_id            timestamp    amount  \
0  f2f3b0fc-b7d7-4d85-b682-cbf07ef77c1a  2025-03-06 12:04:00   7746.03   
1  feccbd1e-7752-4452-a44a-901510252561  2025-03-31 01:37:00  45193.04   
2  bc6a480c-e2e1-434d-830c-1cda3f913038  2025-04-29 22:58:00  33029.71   
3  b832aa2c-77f3-4c4f-9697-9776b8d74abb  2025-02-27 14:52:00  35994.61   
4  0d32c6ef-508e-4f5d-bc64-341a0a9f498d  2025-01-03 22:29:00  32229.73   

  currency            sender_account          receiver_account sender_country  \
0      SEK  SE8902EWFT80524424320740  SE8902ZUIH32054423564254         Sweden   
1      SEK  SE8902GTSI58472323675409  SE8902ERNA19462716907689         Sweden   
2      SEK  SE8902SEZT63377163386089  SE8902LOLQ30361773270969         Sweden   
3      SEK  SE8902MGEE43594824630478  SE8902XIVG18316797897983         Sweden   
4      SEK  SE8902SZWO66860675966705  SE8902SUUS97732018069804         Sweden   

  sender_municipality receiver_country receiver_municipality transac

---
### Rätta felstavningar (currency)

In [42]:
print(df['currency'].unique())

['SEK' 'SKR' 'DKK' 'USD' 'EUR' 'NOK' 'RMB' 'ZAR' 'GBP' 'ZMW' 'JPY']


In [43]:
currency_corrections = {
    'SKR': 'SEK',
    'RMB': 'CNY'  # google sa att rmb är valutan men cny är officiella koden
}
df['currency'] = df['currency'].replace(currency_corrections)
print(df['currency'].unique())

['SEK' 'DKK' 'USD' 'EUR' 'NOK' 'CNY' 'ZAR' 'GBP' 'ZMW' 'JPY']


---
### Fixa timestamp kolumnen (saknade och felaktigt formaterade datum)

In [44]:
df['timestamp_original'] = df['timestamp']

# för att spara en kopia av de ursprungliga datumen innan rensningen börjar, så om något går fel kan man alltid gå tillbaka.

In [93]:
def fix_timestamp(text):
    try:
        return parser.parse(str(text))
    except:
        return None

df['timestamp_cleaned'] = df['timestamp_original'].apply(fix_timestamp)

df['timestamp_cleaned'] = df['timestamp_cleaned'].apply(
    lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if pd.notna(x) else None
)

In [46]:
df['timestamp'] = df['timestamp_cleaned']
df.drop(columns=['timestamp_cleaned'], inplace=True)

In [53]:
mask = df['timestamp'] != df['timestamp_original']
df.loc[mask, ['timestamp_original', 'timestamp']]

,timestamp_original,timestamp
372,20250125 04:48:00,2025-01-25 04:48:00
374,25-04-09 12:12:00,2009-04-25 12:12:00
432,25-01-26 09:01:00,2026-01-25 09:01:00
566,20250215 07:06:00,2025-02-15 07:06:00
658,2025-01-18 16:14,2025-01-18 16:14:00
661,2025-02-16 23:04,2025-02-16 23:04:00
761,2025-05-18 09:52,2025-05-18 09:52:00
814,2025-01-30 23.30,2025-01-30 23:30:00
817,2025-04-01 09.15:00,2025-04-01 09:00:00
821,2025-01-31 24:30:00,2025-02-01 00:30:00


In [48]:
mask = df['timestamp'].isna() & df['timestamp_original'].notna()
to_fix = df.loc[mask, 'timestamp_original']

In [49]:
def smart_fix(ts):
    if pd.isna(ts):
        return None

    ts = str(ts).strip()
    ts = ts.replace('/', '-')
    ts = ts.replace('.', ':')

    if len(ts) <= 10:
        ts += " 00:00:00"

    parts = ts.split(' ')
    if len(parts) == 2 and len(parts[1].split(':')) == 2:
        ts = parts[0] + ' ' + parts[1] + ':00'

    try:
        return pd.to_datetime(ts, errors='raise')
    except:
        pass

    try:
        date_part, time_part = ts.split(' ')
        h, m, *s = time_part.split(':')
        if h == '24':
            h = '00'
            time_fixed = ':'.join([h, m] + (s if s else ['00']))
            dt = pd.to_datetime(date_part, errors='raise') + timedelta(days=1)
            return pd.to_datetime(f"{dt.date()} {time_fixed}")
    except:
        pass

    try:
        date_part, time_part = ts.split(' ')
        y, m, d = map(int, date_part.split('-'))
        d_max = calendar.monthrange(y, m)[1]
        d = min(d, d_max)
        dt = pd.to_datetime(f"{y}-{m:02d}-{d:02d} {time_part}", errors='raise')
        return dt
    except:
        return None

In [50]:
fixed = to_fix.apply(smart_fix)
df.loc[mask, 'timestamp'] = fixed.dt.strftime('%Y-%m-%d %H:%M:%S')

In [51]:
df.loc[mask, ['timestamp_original', 'timestamp']]

,timestamp_original,timestamp
814,2025-01-30 23.30,2025-01-30 23:30:00
821,2025-01-31 24:30:00,2025-02-01 00:30:00
866,2025-04-31 15:17:00,2025-04-30 15:17:00
876,2025-02-30 02:58:00,2025-02-28 02:58:00
884,2025-02-29 01:20:00,2025-02-28 01:20:00
911,2025-02-29 09:00:00,2025-02-28 09:00:00
913,2025-02-30 19:43:00,2025-02-28 19:43:00
1428,2025-01-00 10:48:00,NaN


In [54]:
# Fixa 1428 manuellt (får inte egentligen men det är det närmaste giltliga datumet till 00)
df.loc[1428, 'timestamp'] = pd.to_datetime("2025-01-01 10:48:00")

In [55]:
df.loc[1428, ['timestamp_original', 'timestamp']]

timestamp_original    2025-01-00 10:48:00
timestamp             2025-01-01 10:48:00
Name: 1428, dtype: object

In [56]:
df.dtypes

transaction_id            object
timestamp                 object
amount                   float64
currency                  object
sender_account            object
receiver_account          object
sender_country            object
sender_municipality       object
receiver_country          object
receiver_municipality     object
transaction_type          object
notes                     object
timestamp_original        object
dtype: object

In [57]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

In [58]:
print(df['timestamp'].dtype)

datetime64[ns]


---
### Lösning för nullvärden: Sender och receiver country/municipality

In [63]:
cols = ['sender_country', 'sender_municipality', 'receiver_country', 'receiver_municipality']
df[cols] = df[cols].fillna('Unknown')

In [87]:
print(df.isna().sum())

transaction_id              0
timestamp                   0
amount                      0
currency                    0
sender_account              0
receiver_account            0
sender_country              0
sender_municipality         0
receiver_country            0
receiver_municipality       0
transaction_type            0
notes                    9982
timestamp_original          0
dtype: int64


Alltså lägger "unknown" där det är tomt för att tydligt markera att datan saknas utan att ta bort nånting eller hitta på egna värden, så man inte heller går emot "Accuracy".
Det gör att datan blir iallafall komplett o körbart och går att analysera.

---
### Nullvärden - notes

Hellre behåller nullvärden, att använda "Unknown" lösningen funkar inte lika bra här, då man inte längre kan skilja mellan saknade anteckningar och någon som har skrivit "unknown" som kommentar.

In [91]:
print(df['notes'].isna().sum())

9982
